In [ ]:
# Выделить числовые, бинарные и категориальные признаки.
# Категориальные признаки закодировать в эмбединги. 
# Сначала -1 заменить на 999.
# Числовые признаки нормализовать с помощью GaussRank.

# Все эти данные засунуть в фастаи и обучить полносвязную модель, 
# чтобы получить эмбединги категориальных признаков.

# Сформировать и сохранить однородный датасет в hdf5 (обучающую и 
# тестовую выборки)

# Обучить DAE, в отдельной тетрадке обучить энкодер на аномалии. 
# Значения MSE будет отдельным признаком!

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys; sys.path.append('scripts')
from common import *
from global_common import GaussRankScaler
from models import *
from load_and_proccess_data import *
from visualization import *

In [3]:
# Загружает уже обработанные признаки 
train = pd.read_hdf('../data/train_processed.h5')
test = pd.read_hdf('../data/test_processed.h5')
with open('../data/features_dtypes.pkl', 'rb') as f:
    columns_type_dict = pickle.load(f)
    
# Для генерации признаков нуэно объединить обуч. и тестовую выборки.
test['target'] = np.nan
data = pd.concat([train, test], axis=0)

data.dtypes.value_counts().plot.bar(edgecolor = 'k');
plt.title('Variable Type Distribution');

Из целочисленныъ - 4 бинарных.
Целочисленных переменных  : 232
Переменных с плавающей точкой : 225
Переменным присвоен соответствующий тип.


In [4]:
categorical_features = [
    'os_category', 'device_type', 'service_7_flag_m1', 
    'service_7_flag_m2', 'service_7_flag_m3',
]
data[categorical_features].head(2)

binary_features = [
    'tp_flag', 'block_flag', 'service_2_flag', 'is_obl_center', 
    'is_my_vf', 'service_9_flag_m1', 'service_9_flag_m2', 
    'service_9_flag_m3',
]
data[binary_features].head(2)

# Некоторые числовые признаки мы удалили из-за высокой корреляции
# поэтому фильтруем те что были удалены
numerical_features = [o for o in columns_type_dict['float_cols'] \
                               if o in data.columns]
data[numerical_features].head(2)

procs = [Categorify] 

In [5]:
# Нормализуем магией
numerical_features_df = data[numerical_features]

scaler = GaussRankScaler()
numerical_features_df_ = scaler.fit_transform( numerical_features_df )

data[num_cols] = num_cols_df_

train_ = data.iloc[:train_len+1, :]
test_ = data.iloc[train_len+1:, :]
print(train_.shape, test_.shape)

(70000, 459) (30000, 459)


# Категориальные -> эмбединги

In [ ]:
path_to_data = '../data'

In [ ]:
_, valid_idxs, _, _ = train_test_split(train_df.index, train_df.target, 
                                    stratify=train_df.target,
                                    test_size=0.2, 
                                    random_state=17)

In [ ]:
test = TabularList.from_df(test_df, path=path_to_data, cat_names=cat_names, cont_names=cont_names)

In [ ]:
data = (TabularList.from_df(train_df, path=path_to_data, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(valid_idxs)
                           .label_from_df(cols=dep_var)
                           .add_test(test, label=0)
                           .databunch())

In [ ]:
roc_auc = RocAuc(roc_auc_score)
learn = tabular_learner(data, layers=[1000,500], metrics=[accuracy, roc_auc], emb_drop=0.002, ps=0.3)

In [ ]:
learn.fit_one_cycle(10, 1e-2)

In [6]:
import gc
gc.collect()

21

In [15]:
import tqdm 
import torch
from torch import nn
import torch.nn.functional as F

class DAE_DS(nn.Module):
    def __init__(self, df):
        super(DAE_DS, self).__init__()        
        self.df = df
        
    def __len__(self):
        return self.df.shape[0]
        
    def __getitem__(self, idx):
        x = self.df.iloc[idx, :].values.astype(np.float32)
        x = torch.tensor(x)
        return x, x
        

class DAE(nn.Module):
    def __init__(self, n_columns, n_neurons):
        super(DAE, self).__init__()        
        self.layer1 = nn.Linear(in_features=n_columns, out_features=n_neurons)
        self.layer2 = nn.Linear(in_features=n_neurons, out_features=n_neurons)
        self.layer3 = nn.Linear(in_features=n_neurons, out_features=n_neurons)
        self.output = nn.Linear(in_features=n_neurons, out_features=n_columns)
        
    def forward(self, x):
        l1 = F.relu(self.layer1(x))
        l2 = F.relu(self.layer2(l1))
        l3 = F.relu(self.layer3(l2))
        return self.output(l3)

In [8]:
train_idxs, valid_idxs, _, _ = train_test_split(train_.index, 
                                                train_df.target.values, 
                                                stratify=train_df.target.values,
                                                test_size=0.2, 
                                                random_state=SEED)

train_ds = DAE_DS(train_.iloc[train_idxs, :])
valid_ds = DAE_DS(train_.iloc[valid_idxs, :])

bs = 128
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=bs, shuffle=True)

In [12]:
model_dae = DAE(n_columns=459, n_neurons=1000).cuda()
loss_function = nn.MSELoss()

lr=8e-1
optimizer = torch.optim.SGD(model_dae.parameters(), lr=lr, weight_decay=1e-4, momentum=0.6)

In [10]:
log_lrs, losses = find_lr(model_dae, train_dl, optimizer, loss_function)
plot_lr_find(log_lrs, losses)

In [13]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

In [16]:
epochs=10
for epoch in range(epochs):
    training_loss, validation_loss  = 0.0, 0.0
    for dataset, training in [(train_dl, True),
                              (valid_dl, False)]:
        torch.set_grad_enabled(training)
        model_dae.train(training)
        t = tqdm.tqdm_notebook(dataset)
        
        for batch_idx, (x, y) in enumerate(t):               
                x, y = x.cuda(), y.cuda()
                model_dae.zero_grad()
                scores = model_dae(x)
                loss = loss_function(scores, y)
                if training:
                    loss.backward()
                    training_loss = loss.item()
                    t.set_postfix(epoch=epoch, training_loss=training_loss, refresh=False)
                    optimizer.step()
                    scheduler.step()
                else:
                    validation_loss = loss.item()       
                    t.set_postfix(epoch=epoch, validation_loss=validation_loss, refresh=False)      

KeyboardInterrupt: 

In [ ]:

    
    
    

# model_dae = Sequential()

# model_dae.add(Dense(1000, activation='relu', input_shape=(d.shape[1],))) 
# model_dae.add(Dense(1000, activation='relu'))
# model_dae.add(Dense(1000, activation='relu'))
# model_dae.add(Dense(d.shape[1], activation='linear'))

# model_opt = keras.optimizers.SGD(lr=0.1, decay=1-0.995, momentum=0, nesterov=False)

# model_dae.compile(optimizer=model_opt, loss='mean_squared_error')